In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os,sys
import scanpy as sc 
import pandas as pd
import numpy as np
# import milopy
import scipy
import anndata

In [3]:
# check X stores raw counts 
def _check_counts_in_X(adata):
    return(all(np.random.choice(adata.X.data, 100) % 1 == 0))

def _clean_adata(a):
    ## Make obs_names unique
    a.obs_names = a.obs['dataset_id'].astype('str') + '-' + a.obs_names.astype("str")
    assert _check_counts_in_X(a)

    sc.pp.calculate_qc_metrics(a, inplace=True)
    sc.pp.filter_cells(a, min_counts=1000)
    return(a)

### Prep COVID dataset

In [5]:
h5ad_files_covid = 'PBMC_COVID.subsample500cells.covid.h5ad'

adata_covid = sc.read_h5ad(h5ad_files_covid)
cleaned_covid = _clean_adata(adata_covid)

### Prep Control Dataset

In [6]:
h5ad_files_ctrl = 'PBMC_COVID.subsample500cells.ctrl.h5ad'

adata_ctrl = sc.read_h5ad(h5ad_files_ctrl)
cleaned_ctrl = _clean_adata(adata_ctrl)

## Get Foreground (COVID dataset)

### Expression_matrix_covid and annotations_covid

rows represent individual cells
columns represent genes 

In [7]:
expression_matrix_covid = cleaned_covid.X
annotations_covid = cleaned_covid.obs   # for observations (e.g., cells)
var_annotations_covid = cleaned_covid.var   # for variables (e.g., genes)

In [8]:
# Access other metadata
metadata_covid = cleaned_covid.uns 

In [9]:
print(expression_matrix_covid)

  (0, 75)	1.0
  (0, 97)	1.0
  (0, 110)	1.0
  (0, 161)	1.0
  (0, 205)	1.0
  (0, 214)	1.0
  (0, 247)	1.0
  (0, 314)	1.0
  (0, 325)	1.0
  (0, 339)	2.0
  (0, 366)	1.0
  (0, 389)	1.0
  (0, 393)	2.0
  (0, 416)	1.0
  (0, 438)	1.0
  (0, 455)	1.0
  (0, 458)	1.0
  (0, 504)	1.0
  (0, 542)	1.0
  (0, 586)	1.0
  (0, 614)	1.0
  (0, 697)	1.0
  (0, 700)	2.0
  (0, 718)	1.0
  (0, 774)	1.0
  :	:
  (48082, 24522)	1.0
  (48082, 24524)	1.0
  (48082, 24545)	3.0
  (48082, 24558)	1.0
  (48082, 24580)	1.0
  (48082, 24587)	1.0
  (48082, 24588)	4.0
  (48082, 24614)	1.0
  (48082, 24629)	1.0
  (48082, 24662)	2.0
  (48082, 24663)	8.0
  (48082, 24691)	1.0
  (48082, 24699)	1.0
  (48082, 24700)	9.0
  (48082, 24701)	3.0
  (48082, 24702)	24.0
  (48082, 24703)	13.0
  (48082, 24704)	2.0
  (48082, 24705)	4.0
  (48082, 24706)	10.0
  (48082, 24707)	2.0
  (48082, 24708)	3.0
  (48082, 24709)	1.0
  (48082, 24710)	2.0
  (48082, 24712)	1.0


In [10]:
expression_matrix_covid_dense = expression_matrix_covid.todense()
print(expression_matrix_covid_dense)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [11]:
expression_matrix_covid_array = expression_matrix_covid.toarray()
expression_matrix_covid_df = pd.DataFrame(expression_matrix_covid_array)
# expression_matrix_covid_array.to_csv("expression_matrix_covid_array.csv")

In [12]:
print(annotations_covid)

                                                     sex tissue ethnicity  \
10_1038_s41591_021_01329_2-10_1038_s41591_021_0...  male  blood   unknown   
10_1038_s41591_021_01329_2-10_1038_s41591_021_0...  male  blood   unknown   
10_1038_s41591_021_01329_2-10_1038_s41591_021_0...  male  blood   unknown   
10_1038_s41591_021_01329_2-10_1038_s41591_021_0...  male  blood   unknown   
10_1038_s41591_021_01329_2-10_1038_s41591_021_0...  male  blood   unknown   
...                                                  ...    ...       ...   
10_1038_s41591_021_01329_2-10_1038_s41591_021_0...  male  blood   unknown   
10_1038_s41591_021_01329_2-10_1038_s41591_021_0...  male  blood   unknown   
10_1038_s41591_021_01329_2-10_1038_s41591_021_0...  male  blood   unknown   
10_1038_s41591_021_01329_2-10_1038_s41591_021_0...  male  blood   unknown   
10_1038_s41591_021_01329_2-10_1038_s41591_021_0...  male  blood   unknown   

                                                     disease  \
10_1038_s41

In [13]:
cell_type = annotations_covid['cell_type']

In [14]:
cell_type_unique = set(cell_type)
print(cell_type_unique)

{'CD4-positive, alpha-beta T cell', 'CD34-positive, CD38-negative hematopoietic stem cell', 'B cell', 'naive thymus-derived CD4-positive, alpha-beta T cell', 'gamma-delta T cell', 'dendritic cell', 'monocyte', 'CD14-positive monocyte', 'platelet', 'effector memory CD4-positive, alpha-beta T cell', 'hematopoietic precursor cell', 'IgA plasma cell', 'group 2 innate lymphoid cell, human', 'CD8-positive, alpha-beta T cell', 'myeloid lineage restricted progenitor cell', 'IgG plasma cell', 'ILC1, human', 'IgM plasma cell', 'T follicular helper cell', 'CD14-low, CD16-positive monocyte', 'class switched memory B cell', 'unswitched memory B cell', 'CD16-positive, CD56-dim natural killer cell, human', 'immature B cell', 'central memory CD4-positive, alpha-beta T cell', 'erythrocyte', 'plasmacytoid dendritic cell', 'dendritic cell, human', 'malignant cell', 'natural killer cell', 'CD16-negative, CD56-bright natural killer cell, human', 'T-helper 1 cell', 'naive thymus-derived CD8-positive, alpha-

In [15]:
len(cell_type_unique)

46

In [16]:
cell_type_total = cell_type.value_counts() 
print(cell_type_total)

cell_type
CD14-positive monocyte                                   6803
CD16-positive, CD56-dim natural killer cell, human       6449
central memory CD4-positive, alpha-beta T cell           5382
naive thymus-derived CD4-positive, alpha-beta T cell     4346
naive B cell                                             4043
effector CD8-positive, alpha-beta T cell                 3356
naive thymus-derived CD8-positive, alpha-beta T cell     2505
T follicular helper cell                                 1836
T-helper 22 cell                                         1456
effector memory CD8-positive, alpha-beta T cell          1212
platelet                                                 1194
mature NK T cell                                         1130
gamma-delta T cell                                       1024
CD16-negative, CD56-bright natural killer cell, human     756
mucosal invariant T cell                                  755
class switched memory B cell                              61

In [17]:
cell_type_total = cell_type.value_counts() 
cell_type_greater_than_250 = cell_type_total[cell_type_total > 250]
print(cell_type_greater_than_250)
len(cell_type_greater_than_250)

cell_type
CD14-positive monocyte                                   6803
CD16-positive, CD56-dim natural killer cell, human       6449
central memory CD4-positive, alpha-beta T cell           5382
naive thymus-derived CD4-positive, alpha-beta T cell     4346
naive B cell                                             4043
effector CD8-positive, alpha-beta T cell                 3356
naive thymus-derived CD8-positive, alpha-beta T cell     2505
T follicular helper cell                                 1836
T-helper 22 cell                                         1456
effector memory CD8-positive, alpha-beta T cell          1212
platelet                                                 1194
mature NK T cell                                         1130
gamma-delta T cell                                       1024
CD16-negative, CD56-bright natural killer cell, human     756
mucosal invariant T cell                                  755
class switched memory B cell                              61

28

### Get the Foreground X (COVID)

In [18]:
annotations_covid_df = pd.DataFrame(annotations_covid)
annotations_covid_df_copy = annotations_covid_df

# Reset the index to get numeric indices
annotations_covid_df_copy.reset_index(drop=True, inplace=True)

annotations_covid_df_copy[annotations_covid_df_copy['cell_type'] == 'dendritic cell'].index

Index([ 6656,  6661, 11088, 11243, 13509, 14630, 18324, 21726, 22677, 24405,
       24499, 24507, 24581, 25135, 25584, 26376, 27008, 28504, 28823, 28876,
       30273, 34569, 34837, 35525, 36196, 36683, 42346, 42460, 44282, 44729,
       45355, 46614, 47239, 47567],
      dtype='int64')

In [19]:
filtered_annotations_covid = annotations_covid_df_copy[annotations_covid_df_copy['cell_type'].isin(cell_type_greater_than_250.index)]

filtered_covid_indices = filtered_annotations_covid.index.to_list()
print(len(filtered_covid_indices))

46958


In [20]:
filter_expression_matrix_covid_df = expression_matrix_covid_df.iloc[filtered_covid_indices]
print(filter_expression_matrix_covid_df.shape)

(46958, 24727)


In [21]:
foregroundX = filter_expression_matrix_covid_df
# foregroundX.to_csv('COVID_design_foregroundX.csv')

### Get the foreground Y

In [22]:
# pd.set_option('display.max_columns', None)
print(filtered_annotations_covid.head())

    sex tissue ethnicity   disease                           assay  \
0  male  blood   unknown  COVID-19  10x 3' transcription profiling   
1  male  blood   unknown  COVID-19  10x 3' transcription profiling   
2  male  blood   unknown  COVID-19  10x 3' transcription profiling   
3  male  blood   unknown  COVID-19  10x 3' transcription profiling   
4  male  blood   unknown  COVID-19  10x 3' transcription profiling   

  assay_ontology_term_id sample_id donor_id                  dataset_id  \
0            EFO:0030003       AP1      AP1  10_1038_s41591_021_01329_2   
1            EFO:0030003       AP1      AP1  10_1038_s41591_021_01329_2   
2            EFO:0030003       AP1      AP1  10_1038_s41591_021_01329_2   
3            EFO:0030003       AP1      AP1  10_1038_s41591_021_01329_2   
4            EFO:0030003       AP1      AP1  10_1038_s41591_021_01329_2   

          development_stage  \
0  sixth decade human stage   
1  sixth decade human stage   
2  sixth decade human stage   
3  s

In [23]:
filtered_cell_type = filtered_annotations_covid['cell_type']
print(len(filtered_cell_type))
unique_filtered_cell_type = set(filtered_cell_type)
print(len(unique_filtered_cell_type))

46958
28


In [26]:
from sklearn.preprocessing import LabelEncoder
pd.reset_option('display.max_columns')
pd.reset_option('display.max_rows')

label_encoder = LabelEncoder()

filtered_annotations_covid.loc[:, 'cell_type_numeric'] = label_encoder.fit_transform(filtered_annotations_covid['cell_type']) + 1
print(filtered_annotations_covid['cell_type_numeric'])

0         7
1         9
2         4
3        23
4         4
         ..
48078     2
48079    19
48080     4
48081    11
48082     2
Name: cell_type_numeric, Length: 46958, dtype: int64


#### Get encoding pairings

In [27]:
label_encoding_df = pd.DataFrame({
    'cell_type': label_encoder.classes_,
    'cell_type_numeric': range(1, len(label_encoder.classes_) + 1)
})

print(label_encoding_df)

                                            cell_type  cell_type_numeric
0                                              B cell                  1
1                              CD14-positive monocyte                  2
2   CD16-negative, CD56-bright natural killer cell...                  3
3   CD16-positive, CD56-dim natural killer cell, h...                  4
4                                     IgA plasma cell                  5
5                                     IgG plasma cell                  6
6                            T follicular helper cell                  7
7                                    T-helper 22 cell                  8
8      central memory CD4-positive, alpha-beta T cell                  9
9                        class switched memory B cell                 10
10                              dendritic cell, human                 11
11           effector CD8-positive, alpha-beta T cell                 12
12    effector memory CD4-positive, alpha-beta T ce

### Output foreground Y

In [29]:
cell_type_Y = filtered_annotations_covid['cell_type_numeric']
# print(cell_type_Y)
foregroundY = cell_type_Y.to_frame()
print(type(foregroundY))
# foregroundY.to_csv('COVID_design_foregroundY.csv')

<class 'pandas.core.frame.DataFrame'>


In [30]:
print(var_annotations_covid)

                feature_biotype  n_cells_by_counts  mean_counts  \
feature_id                                                        
ENSG00000243485            gene                  0     0.000000   
ENSG00000238009            gene                 39     0.000811   
ENSG00000239945            gene                  0     0.000000   
ENSG00000239906            gene                  0     0.000000   
ENSG00000229905            gene                  0     0.000000   
...                         ...                ...          ...   
ENSG00000277196            gene                  2     0.000042   
ENSG00000278384            gene                123     0.002641   
ENSG00000277856            gene                187     0.189672   
ENSG00000275063            gene                235     0.129859   
ENSG00000271254            gene                434     0.009234   

                 log1p_mean_counts  pct_dropout_by_counts  total_counts  \
feature_id                                           

## Get Background (Control dataset)

In [31]:
expression_matrix_control = cleaned_ctrl.X
annotations_control = cleaned_ctrl.obs   # for observations (e.g., cells)
var_annotations_control = cleaned_ctrl.var   # for variables (e.g., genes)

#### Process expression_matrix_control

In [32]:
print(expression_matrix_control)

  (0, 26)	1.0
  (0, 28)	1.0
  (0, 42)	1.0
  (0, 43)	1.0
  (0, 75)	1.0
  (0, 97)	1.0
  (0, 110)	9.0
  (0, 116)	1.0
  (0, 141)	1.0
  (0, 150)	3.0
  (0, 166)	3.0
  (0, 168)	1.0
  (0, 287)	1.0
  (0, 289)	1.0
  (0, 290)	1.0
  (0, 314)	1.0
  (0, 325)	4.0
  (0, 339)	24.0
  (0, 356)	1.0
  (0, 358)	3.0
  (0, 360)	1.0
  (0, 369)	1.0
  (0, 372)	1.0
  (0, 393)	5.0
  (0, 395)	3.0
  :	:
  (14425, 24520)	1.0
  (14425, 24527)	1.0
  (14425, 24533)	1.0
  (14425, 24558)	2.0
  (14425, 24580)	2.0
  (14425, 24588)	1.0
  (14425, 24614)	1.0
  (14425, 24627)	1.0
  (14425, 24629)	1.0
  (14425, 24659)	1.0
  (14425, 24661)	1.0
  (14425, 24663)	2.0
  (14425, 24700)	6.0
  (14425, 24701)	5.0
  (14425, 24702)	45.0
  (14425, 24703)	33.0
  (14425, 24704)	6.0
  (14425, 24705)	13.0
  (14425, 24706)	27.0
  (14425, 24707)	7.0
  (14425, 24708)	18.0
  (14425, 24709)	1.0
  (14425, 24710)	9.0
  (14425, 24711)	2.0
  (14425, 24712)	28.0


In [33]:
expression_matrix_control_array = expression_matrix_control.toarray()
expression_matrix_control_df = pd.DataFrame(expression_matrix_control_array)

In [34]:
print(annotations_control)

                                                     sex tissue ethnicity  \
10_1038_s41591_021_01329_2-10_1038_s41591_021_0...  male  blood   unknown   
10_1038_s41591_021_01329_2-10_1038_s41591_021_0...  male  blood   unknown   
10_1038_s41591_021_01329_2-10_1038_s41591_021_0...  male  blood   unknown   
10_1038_s41591_021_01329_2-10_1038_s41591_021_0...  male  blood   unknown   
10_1038_s41591_021_01329_2-10_1038_s41591_021_0...  male  blood   unknown   
...                                                  ...    ...       ...   
10_1038_s41591_021_01329_2-10_1038_s41591_021_0...  male  blood   unknown   
10_1038_s41591_021_01329_2-10_1038_s41591_021_0...  male  blood   unknown   
10_1038_s41591_021_01329_2-10_1038_s41591_021_0...  male  blood   unknown   
10_1038_s41591_021_01329_2-10_1038_s41591_021_0...  male  blood   unknown   
10_1038_s41591_021_01329_2-10_1038_s41591_021_0...  male  blood   unknown   

                                                   disease  \
10_1038_s4159

#### background (Control) has 41 cell types

In [35]:
cell_type_control = annotations_control['cell_type']
cell_type_control_unique = set(cell_type_control)
print(len(cell_type_control_unique))

41


In [36]:
cell_type_control_total = cell_type_control.value_counts()
cell_type_control_total

cell_type
CD16-positive, CD56-dim natural killer cell, human       2117
naive thymus-derived CD4-positive, alpha-beta T cell     1838
naive B cell                                             1326
CD14-positive monocyte                                   1183
central memory CD4-positive, alpha-beta T cell           1182
naive thymus-derived CD8-positive, alpha-beta T cell     1092
effector CD8-positive, alpha-beta T cell                  813
T-helper 22 cell                                          723
effector memory CD8-positive, alpha-beta T cell           704
gamma-delta T cell                                        483
mature NK T cell                                          457
mucosal invariant T cell                                  391
platelet                                                  373
CD16-negative, CD56-bright natural killer cell, human     359
class switched memory B cell                              247
immature B cell                                           18

In [37]:
cell_type_control_greater_than_250 = cell_type_control_total[cell_type_control_total > 250] 
print(cell_type_control_greater_than_250)
len(cell_type_control_greater_than_250)

cell_type
CD16-positive, CD56-dim natural killer cell, human       2117
naive thymus-derived CD4-positive, alpha-beta T cell     1838
naive B cell                                             1326
CD14-positive monocyte                                   1183
central memory CD4-positive, alpha-beta T cell           1182
naive thymus-derived CD8-positive, alpha-beta T cell     1092
effector CD8-positive, alpha-beta T cell                  813
T-helper 22 cell                                          723
effector memory CD8-positive, alpha-beta T cell           704
gamma-delta T cell                                        483
mature NK T cell                                          457
mucosal invariant T cell                                  391
platelet                                                  373
CD16-negative, CD56-bright natural killer cell, human     359
Name: count, dtype: int64


14

### Get the Background X (Control) 

In [38]:
annotation_control_df = pd.DataFrame(annotations_control)
annotation_control_df_copy = annotation_control_df
annotation_control_df.shape

(14426, 20)

#### Reset the index to get numeric indices

In [40]:
annotation_control_df_copy.reset_index(drop=True, inplace=True)

filtered_annotations_control = annotation_control_df_copy[annotation_control_df_copy['cell_type'].isin(cell_type_control_greater_than_250.index)]

#### check the indices for cell_type less than 250 (to ensure the filtered_control_indices remove these indices)

In [41]:
annotation_control_df_copy[annotation_control_df_copy['cell_type'] == 'plasmablast'].index

Index([ 1239,  2216,  2570,  6617,  9012,  9048,  9651,  9657, 10096, 10164,
       10657, 11574, 12442, 13396, 14210],
      dtype='int64')

In [42]:
filtered_control_indices = filtered_annotations_control.index.to_list()
print(len(filtered_control_indices))

13041


In [43]:
filter_expression_matrix_control_df = expression_matrix_control_df.iloc[filtered_control_indices]
print(filter_expression_matrix_control_df.shape)

(13041, 24727)


In [44]:
backgroundX = filter_expression_matrix_control_df
# backgroundX.to_csv('COVID_design_backgroundX.csv')

### Get the background Y

In [45]:
# pd.set_option('display.max_rows', None)
print(filtered_annotations_control['cell_type'])

0        CD16-negative, CD56-bright natural killer cell...
1                                                 platelet
2                                                 platelet
4                                                 platelet
5        CD16-positive, CD56-dim natural killer cell, h...
                               ...                        
14420             effector CD8-positive, alpha-beta T cell
14421                                         naive B cell
14423             effector CD8-positive, alpha-beta T cell
14424                                             platelet
14425                                     T-helper 22 cell
Name: cell_type, Length: 13041, dtype: category
Categories (41, object): ['B cell', 'CD14-low, CD16-positive monocyte', 'CD14-positive monocyte', 'CD16-negative, CD56-bright natural killer cel..., ..., 'plasmacytoid dendritic cell', 'platelet', 'regulatory T cell', 'unswitched memory B cell']


In [46]:
filtered_cell_type_control = filtered_annotations_control['cell_type'] 
print(len(filtered_cell_type_control))
unique_filtered_cell_type_control = set(filtered_cell_type_control)
print(len(unique_filtered_cell_type_control))

13041
14


In [47]:
print(filtered_cell_type_control)

0        CD16-negative, CD56-bright natural killer cell...
1                                                 platelet
2                                                 platelet
4                                                 platelet
5        CD16-positive, CD56-dim natural killer cell, h...
                               ...                        
14420             effector CD8-positive, alpha-beta T cell
14421                                         naive B cell
14423             effector CD8-positive, alpha-beta T cell
14424                                             platelet
14425                                     T-helper 22 cell
Name: cell_type, Length: 13041, dtype: category
Categories (41, object): ['B cell', 'CD14-low, CD16-positive monocyte', 'CD14-positive monocyte', 'CD16-negative, CD56-bright natural killer cel..., ..., 'plasmacytoid dendritic cell', 'platelet', 'regulatory T cell', 'unswitched memory B cell']


In [49]:
from sklearn.preprocessing import LabelEncoder

label_encoder2 = LabelEncoder()

filtered_annotations_control.loc[:, 'cell_type_numeric'] = label_encoder2.fit_transform(filtered_annotations_control['cell_type']) + 1 
print(filtered_annotations_control['cell_type_numeric'])

0         2
1        14
2        14
4        14
5         3
         ..
14420     6
14421    11
14423     6
14424    14
14425     4
Name: cell_type_numeric, Length: 13041, dtype: int64


In [50]:
label_encoding_control_df = pd.DataFrame({
    'cell_type': label_encoder2.classes_,
    'cell_type_numeric': range(1, len(label_encoder2.classes_) + 1)
})

print(label_encoding_control_df)

                                            cell_type  cell_type_numeric
0                              CD14-positive monocyte                  1
1   CD16-negative, CD56-bright natural killer cell...                  2
2   CD16-positive, CD56-dim natural killer cell, h...                  3
3                                    T-helper 22 cell                  4
4      central memory CD4-positive, alpha-beta T cell                  5
5            effector CD8-positive, alpha-beta T cell                  6
6     effector memory CD8-positive, alpha-beta T cell                  7
7                                  gamma-delta T cell                  8
8                                    mature NK T cell                  9
9                            mucosal invariant T cell                 10
10                                       naive B cell                 11
11  naive thymus-derived CD4-positive, alpha-beta ...                 12
12  naive thymus-derived CD8-positive, alpha-beta .

### Output background Y

In [52]:
cell_type_Yt = filtered_annotations_control['cell_type_numeric']
# print(cell_type_Yt)
backgroundY = cell_type_Yt.to_frame()
# print(backgroundY)

# backgroundY.to_csv('COVID_design_backgroundY.csv')

In [53]:
print(var_annotations_control)

                feature_biotype  n_cells_by_counts  mean_counts  \
feature_id                                                        
ENSG00000243485            gene                  1     0.000069   
ENSG00000238009            gene                 13     0.000901   
ENSG00000239945            gene                  0     0.000000   
ENSG00000239906            gene                  0     0.000000   
ENSG00000229905            gene                  0     0.000000   
...                         ...                ...          ...   
ENSG00000277196            gene                  1     0.000069   
ENSG00000278384            gene                 47     0.003327   
ENSG00000277856            gene                 33     0.219881   
ENSG00000275063            gene                 46     0.040482   
ENSG00000271254            gene                199     0.014210   

                 log1p_mean_counts  pct_dropout_by_counts  total_counts  \
feature_id                                           